## Discover 802.15.4 Jamming

#### Radio settings

If you are running this training at the same time as other people on the testbed, it is a good idea to change the default radio configuration to avoid too much collision with others.

Use the following cell to give you random values for channel and PAN ID.

In [24]:
import os,binascii,random
pan_id = binascii.b2a_hex(os.urandom(2)).decode()
channel = random.randint(11, 26)
print('Use CHANNEL={}, PAN_ID=0x{}'.format(channel, pan_id))

Use CHANNEL=25, PAN_ID=0x9122


For these values to be taken into account you can set an environment variable for the notebook. Modify the values in the cell below with those obtained and execute it.

In [40]:
%env CHANNEL=26

env: CHANNEL=26


In [39]:
%env PAN_ID=0x0001

env: PAN_ID=0x0001


### Compile the RIOT firmware

Here you will use the `default` example from RIOT.

Let's define a environment variable to shorten the command lines.

In [ ]:
%env APP_DIR = .

Compile the example:

In [ ]:
!make clean

In [41]:
!make -C $APP_DIR BOARD=iotlab-m3 DEFAULT_CHANNEL=$CHANNEL DEFAULT_PAN_ID=$PAN_ID

make: Entering directory '/home/jovyan/work/training/CprE537_Jamming_AntiJamming/Jammer'
Building application "hello-jammer" for "iotlab-m3" with MCU "stm32".

"make" -C /home/jovyan/work/training/CprE537_Jamming_AntiJamming/Jammer/RIOT/boards/iotlab-m3
"make" -C /home/jovyan/work/training/CprE537_Jamming_AntiJamming/Jammer/RIOT/boards/common/iotlab
"make" -C /home/jovyan/work/training/CprE537_Jamming_AntiJamming/Jammer/RIOT/core
"make" -C /home/jovyan/work/training/CprE537_Jamming_AntiJamming/Jammer/RIOT/cpu/stm32
"make" -C /home/jovyan/work/training/CprE537_Jamming_AntiJamming/Jammer/RIOT/cpu/cortexm_common
"make" -C /home/jovyan/work/training/CprE537_Jamming_AntiJamming/Jammer/RIOT/cpu/cortexm_common/periph
"make" -C /home/jovyan/work/training/CprE537_Jamming_AntiJamming/Jammer/RIOT/cpu/stm32/periph
"make" -C /home/jovyan/work/training/CprE537_Jamming_AntiJamming/Jammer/RIOT/cpu/stm32/stmclk
"make" -C /home/jovyan/work/training/CprE537_Jamming_AntiJamming/Jammer/RIOT/cpu/stm32/vecto

### Submit an experiment on IoT-LAB

1. Choose your site (grenoble|lille|saclay|strasbourg):

In [42]:
%env SITE = strasbourg

env: SITE=strasbourg


2. Submit an experiment with three nodes and the compiled firware:

In [44]:
!iotlab-experiment submit -n "riot-802154" -d 60 -l 1,archi=m3:at86rf231+site=strasbourg,$APP_DIR/bin/iotlab-m3/hello-jammer.elf,joe_cool

{
    "id": 369515
}


3. Wait for the experiment to be in the Running state:

In [46]:
!iotlab-experiment wait --timeout 30 --cancel-on-timeout

Waiting that experiment 369515 gets in state Running
"Running"


**Note:** If the command above returns the message `Timeout reached, cancelling experiment <exp_id>`, try to re-submit your experiment later or try on another site.

4. Check the nodes allocated to the experiment (filtered by network address)

In [47]:
!iotlab-experiment --jmespath="items[*].network_address | sort(@)" get --nodes

[
    "m3-1.strasbourg.iot-lab.info",
    "m3-12.strasbourg.iot-lab.info",
    "m3-13.strasbourg.iot-lab.info",
    "m3-14.strasbourg.iot-lab.info",
    "m3-15.strasbourg.iot-lab.info",
    "m3-16.strasbourg.iot-lab.info",
    "m3-17.strasbourg.iot-lab.info",
    "m3-18.strasbourg.iot-lab.info",
    "m3-19.strasbourg.iot-lab.info",
    "m3-2.strasbourg.iot-lab.info",
    "m3-20.strasbourg.iot-lab.info",
    "m3-21.strasbourg.iot-lab.info",
    "m3-22.strasbourg.iot-lab.info",
    "m3-23.strasbourg.iot-lab.info",
    "m3-24.strasbourg.iot-lab.info",
    "m3-25.strasbourg.iot-lab.info",
    "m3-26.strasbourg.iot-lab.info",
    "m3-27.strasbourg.iot-lab.info",
    "m3-28.strasbourg.iot-lab.info",
    "m3-29.strasbourg.iot-lab.info",
    "m3-3.strasbourg.iot-lab.info",
    "m3-30.strasbourg.iot-lab.info",
    "m3-31.strasbourg.iot-lab.info",
    "m3-32.strasbourg.iot-lab.info",
    "m3-33.strasbourg.iot-lab.info",
    "m3-34.strasbourg.iot-lab.info",
    "m3-35.strasbourg.iot-lab.info",
  

From here you will interact with the nodes. To this end, you need to connect to node's serial link.

5. For each of the three nodes open a Jupyter terminal (use `File > New > Terminal`), use the following command replacing `<id>` and `<site>` with the right values:

On the two other nodes terminals you should see that the radio packet has been received:
```
PKTDUMP: data received:
~~ SNIP  0 - size:   5 byte, type: NETTYPE_UNDEF (0)
00000000  68  65  6C  6C  6F
~~ SNIP  1 - size:  12 byte, type: NETTYPE_NETIF (-1)
if_pid: 4  rssi: -55  lqi: 255
flags: BROADCAST 
src_l2addr: 11:15
dst_l2addr: FF:FF
~~ PKT    -  2 snips, total size:  17 byte
```
What does this output tell you ?
- SNIP 0 = payload = 5 bytes `68  65  6C  6C  6F` for 5 characters ['h','e','l','l','o']
- SNIP 1 = header ?
  - `if_pid: 4  rssi: -55  lqi: 255` gives radio network interface informations: Received Signal Strength Indication (RSSI) and  the Link Quality Indicator (LQI)
  - `src_l2addr: 11:15` gives the address of the sender
  - `dst12_addr: FF:FF` is the broadcast destination address, which corresponds well to the `flags: BROADCAST`


3. Send now a packet from the first node to the second. Specifying the radio interface of the sender and replace `<long_hwaddr>` with the address of the destination:

You should have a similar output in the destination node's terminal:
```
PKTDUMP: data received:
~~ SNIP  0 - size:   5 byte, type: NETTYPE_UNDEF (0)
00000000  68  65  6C  6C  6F
~~ SNIP  1 - size:  12 byte, type: NETTYPE_NETIF (-1)
if_pid: 4  rssi: -55  lqi: 255
flags: 0x0
src_l2addr: 11:15
dst_l2addr: 2A:AC:F9:65:10:6B:11:15
~~ PKT    -  2 snips, total size:  17 byte
```

- SNIP 0 remains the same since you send the same text message
- SNIP 1 contains the destination address, which correspond well to the node's address whom print this output


### Free up the resources

Since you finished the training, stop your experiment to free up the experiment nodes:

In [ ]:
!iotlab-experiment stop 

The serial link connection through SSH will be closed automatically.